In [ ]:
%load_ext autoreload
%autoreload 2
import json
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import shape, Point
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate, KFold
from sklearn.metrics import r2_score
import seaborn as sns
from scipy.stats import pearsonr

In [ ]:
# Set Matplotlib parameters for better visualization
plt.rcParams['figure.figsize'] = (14, 8)
plt.style.use('seaborn-v0_8-whitegrid')
pd.set_option('future.no_silent_downcasting', True)

In [ ]:
# Colors for consistent city representation
PARIS_COLOR = 'blue'
LONDON_COLOR = 'green'

In [ ]:
# ----------------------------------------------------------------------
# GLOBAL PARAMETERS FOR london_rentals.xls
# ----------------------------------------------------------------------

In [ ]:
LONDON_START_YEAR = 2012
LONDON_END_YEAR = 2019

In [ ]:
# ----------------------------------------------------------------------
# UTILITY FUNCTIONS
# ----------------------------------------------------------------------

In [ ]:
def create_point_from_coords(row, lon_idx, lat_idx):
    """Create a shapely Point from longitude and latitude columns"""
    try:
        lon, lat = row.iloc[lon_idx], row.iloc[lat_idx]
        return Point(lon, lat)
    except Exception as e:
        print(f"Error creating point: {e}")
        return None

def parse_paris_coords(coord_str):
    """Parse coordinate string from Paris data format"""
    try:
        lat_str, lon_str = coord_str.split(',')
        lat = float(lat_str.strip())
        lon = float(lon_str.strip())
        return Point(lon, lat)
    except Exception as e:
        print(f"Error parsing coordinates: {e}")
        return None
        
def convert_geojson_to_shape(geo_str):
    """Convert GeoJSON string to shapely geometry object"""
    try:
        geo_dict = json.loads(geo_str)
        return shape(geo_dict)
    except Exception as e:
        print(f"Error converting geometry: {e}")
        return None

def fit_polynomial_models(X, Y, max_degree=5):
    """Fit polynomial models of different degrees and perform cross-validation"""
    degrees = range(1, max_degree + 1)
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    cv_results_list = []

    for deg in degrees:
        poly = PolynomialFeatures(degree=deg, include_bias=False)
        X_poly = poly.fit_transform(X)
        lr = LinearRegression()
        cv_results = cross_validate(lr, X_poly, Y, cv=kf, 
                                   scoring='neg_mean_squared_error', 
                                   return_train_score=True)
        train_mse = -np.mean(cv_results['train_score'])
        test_mse = -np.mean(cv_results['test_score'])
        cv_results_list.append({
            'degree': deg, 
            'train_mse': train_mse, 
            'test_mse': test_mse
        })
    
    return pd.DataFrame(cv_results_list)
### PLOTS
# For the scatter plot of Airbnb densities
def plot_airbnb_density_scatter_comparison(paris_data, london_data):
    """Create side-by-side scatter plots comparing Airbnb density vs price changes"""
    fig, axes = plt.subplots(1, 2, figsize=(18, 8))
    
    # Paris scatter plot
    axes[0].scatter(paris_data['price_increase'], paris_data['airbnb_density'], 
                alpha=0.7, color=PARIS_COLOR)
    axes[0].set_xlabel('Rental Price Increase (2024 - 2019) [€/m²]')
    axes[0].set_ylabel('Airbnb Density (listings/km²)')
    axes[0].set_title('Paris: Airbnb Density vs. Rental Price Increase')
    axes[0].grid(True, alpha=0.3)
    
    # London scatter plot
    axes[1].scatter(london_data['price_change'], london_data['airbnb_density'], 
                alpha=0.7, color=LONDON_COLOR)
    axes[1].set_xlabel(f'Rental Price Change ({LONDON_END_YEAR}–{LONDON_START_YEAR})')
    axes[1].set_ylabel('Airbnb Density (listings/km²)')
    axes[1].set_title('London: Airbnb Density vs. Rental Price Change')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig, axes

secret_observation = "In London, Airbnb densities are even more strongly correlated with actual rental prices than with price increases !"

# For the bias-variance trade-off plots
def plot_bias_variance_tradeoff_comparison(paris_cv_results, london_cv_results):
    """Plot bias-variance tradeoff curves for both cities side by side"""
    fig, axes = plt.subplots(1, 2, figsize=(18, 8))
    
    # Paris curves
    axes[0].plot(paris_cv_results['degree'], paris_cv_results['train_mse'], 
             marker='o', color=PARIS_COLOR, linestyle='-', label='Training MSE')
    axes[0].plot(paris_cv_results['degree'], paris_cv_results['test_mse'], 
             marker='o', color=PARIS_COLOR, linestyle='--', label='Validation MSE')
    axes[0].set_xlabel('Polynomial Degree')
    axes[0].set_ylabel('Mean Squared Error')
    axes[0].set_title('Paris: Bias-Variance Trade-Off')
    axes[0].set_yscale('log')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # London curves
    axes[1].plot(london_cv_results['degree'], london_cv_results['train_mse'], 
             marker='s', color=LONDON_COLOR, linestyle='-', label='Training MSE')
    axes[1].plot(london_cv_results['degree'], london_cv_results['test_mse'], 
             marker='s', color=LONDON_COLOR, linestyle='--', label='Validation MSE')
    axes[1].set_xlabel('Polynomial Degree')
    axes[1].set_ylabel('Mean Squared Error')
    axes[1].set_title('London: Bias-Variance Trade-Off')
    axes[1].set_yscale('log')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig, axes

# For polynomial regression plots
def plot_polynomial_regression_comparison(paris_X, paris_Y, paris_poly, paris_model, paris_r, paris_deg,
                                         london_X, london_Y, london_poly, london_model, london_r, london_deg):
    """Plot polynomial regression curves for both cities side by side"""
    fig, axes = plt.subplots(1, 2, figsize=(18, 8))
    
    # Paris regression
    paris_X_range = np.linspace(paris_X.min(), paris_X.max(), 100).reshape(-1, 1)
    paris_X_range_poly = paris_poly.transform(paris_X_range)
    paris_Y_pred = paris_model.predict(paris_X_range_poly)
    
    axes[0].scatter(paris_X, paris_Y, alpha=0.7, color=PARIS_COLOR, label='Neighborhoods')
    axes[0].plot(paris_X_range, paris_Y_pred, color=PARIS_COLOR, linestyle='--', 
             label=f'Polynomial Regression (deg {paris_deg}, R = {paris_r:.2f})')
    axes[0].set_xlabel('Rental Price Increase (2024 - 2019) [€/m²]')
    axes[0].set_ylabel('Airbnb Density (listings/km²)')
    axes[0].set_title('Paris: Airbnb Density vs. Rental Price Increase')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # London regression
    london_X_range = np.linspace(london_X.min(), london_X.max(), 100).reshape(-1, 1)
    london_X_range_poly = london_poly.transform(london_X_range)
    london_Y_pred = london_model.predict(london_X_range_poly)
    
    axes[1].scatter(london_X, london_Y, alpha=0.7, color=LONDON_COLOR, label='Neighborhoods')
    axes[1].plot(london_X_range, london_Y_pred, color=LONDON_COLOR, linestyle='--', 
             label=f'Polynomial Regression (deg {london_deg}, R = {london_r:.2f})')
    axes[1].set_xlabel(f'Rental Price Change ({LONDON_END_YEAR}–{LONDON_START_YEAR})')
    axes[1].set_ylabel('Airbnb Density (listings/km²)')
    axes[1].set_title('London: Airbnb Density vs. Rental Price Change')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig, axes

# For density distribution as bar chart
def plot_density_distribution_comparison(paris_data, london_data):
    """Create side-by-side histograms for Airbnb density distributions"""
    fig, axes = plt.subplots(1, 2, figsize=(18, 8))
    
    # Paris density distribution
    sns.histplot(
        paris_data['airbnb_density'], 
        kde=False, 
        ax=axes[0], 
        color=PARIS_COLOR,
        stat='density',
        bins=15,
        alpha=0.7
    )
    axes[0].set_title('Paris: Airbnb Density Distribution', fontsize=14)
    axes[0].set_xlabel('Airbnb Density (listings/km²)', fontsize=12)
    axes[0].set_ylabel('Density', fontsize=12)
    axes[0].grid(True, alpha=0.3)
    
    # London density distribution
    sns.histplot(
        london_data['airbnb_density'], 
        kde=False, 
        ax=axes[1], 
        color=LONDON_COLOR,
        stat='density',
        bins=15,
        alpha=0.7
    )
    axes[1].set_title('London: Airbnb Density Distribution', fontsize=14)
    axes[1].set_xlabel('Airbnb Density (listings/km²)', fontsize=12)
    axes[1].set_ylabel('Density', fontsize=12)
    axes[1].grid(True, alpha=0.3)
    
    # Improve layout
    plt.tight_layout()
    return fig, axes

def fit_best_model(X, Y, cv_results_df):
    """Fit the best polynomial model based on cross-validation results"""
    best_row = cv_results_df.loc[cv_results_df['test_mse'].idxmin()]
    best_deg = int(best_row['degree'])
    
    poly_best = PolynomialFeatures(degree=best_deg, include_bias=False)
    X_poly_best = poly_best.fit_transform(X)
    lr_best = LinearRegression()
    lr_best.fit(X_poly_best, Y)
    r2_best = lr_best.score(X_poly_best, Y)
    r_best = np.sqrt(r2_best) if r2_best >= 0 else 0
    
    return best_deg, poly_best, lr_best, r_best


def plot_boxplot_comparison(paris_df, london_df):
    """Create side-by-side boxplots for both cities"""
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Paris boxplot
    sns.boxplot(x='price_increase_bin', y='airbnb_density', data=paris_df, ax=axes[0], color=PARIS_COLOR)
    axes[0].set_title("Paris: Distribution of Airbnb Density by Price Increase Bins")
    axes[0].set_xlabel("Rental Price Increase (2024 - 2019) [€/m²] (Binned)")
    axes[0].set_ylabel("Airbnb Density (listings per km²)")
    axes[0].tick_params(axis='x', rotation=45)
    
    # London boxplot
    sns.boxplot(x='price_bin', y='airbnb_density', data=london_df, ax=axes[1], color=LONDON_COLOR)
    axes[1].set_title(f"London: Distribution of Airbnb Density by Price Change Bins")
    axes[1].set_xlabel(f"Price Change ({LONDON_END_YEAR}–{LONDON_START_YEAR}) Quintile")
    axes[1].set_ylabel("Airbnb Density (listings per km²)")
    axes[1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    return fig, axes

def plot_quadratic_fit_comparison(paris_df, london_df, paris_best_deg, london_best_deg):
    """Plot polynomial fits on binned data for both cities with R² values using best degree"""
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Paris polynomial fit
    paris_bin_stats = paris_df.groupby('price_increase_bin', observed=False).agg({
        'price_increase': 'mean', 
        'airbnb_density': 'median'
    }).reset_index()
    
    paris_X = paris_bin_stats['price_increase'].values.reshape(-1, 1)
    paris_y = paris_bin_stats['airbnb_density'].values
    
    paris_poly = PolynomialFeatures(degree=paris_best_deg, include_bias=False)
    paris_X_poly = paris_poly.fit_transform(paris_X)
    paris_model = LinearRegression()
    paris_model.fit(paris_X_poly, paris_y)
    
    # Calculate R² for Paris model
    paris_y_pred = paris_model.predict(paris_X_poly)
    paris_r2 = r2_score(paris_y, paris_y_pred)
    
    paris_X_curve = np.linspace(paris_df['price_increase'].min(), paris_df['price_increase'].max(), 100).reshape(-1, 1)
    paris_y_curve = paris_model.predict(paris_poly.transform(paris_X_curve))
    
    axes[0].scatter(paris_X, paris_y, color=PARIS_COLOR, s=80, label='Bin Medians')
    axes[0].plot(paris_X_curve, paris_y_curve, color=PARIS_COLOR, linestyle='--', label=f'Degree {paris_best_deg} Fit')
    
    # Add bin edge vertical lines for Paris
    paris_bin_edges = pd.unique(paris_df['price_increase_bin'].cat.categories.right.values[:-1])
    for edge in paris_bin_edges:
        axes[0].axvline(x=edge, color='gray', linestyle=':', alpha=0.5)
    
    axes[0].set_title(f"Paris: Degree {paris_best_deg} Regression on Binned Data (R² = {paris_r2:.3f})")
    axes[0].set_xlabel("Mean Price Increase (2024 - 2019) (€/m²)")
    axes[0].set_ylabel("Median Airbnb Density (listings/km²)")
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # London polynomial fit
    london_bin_stats = london_df.groupby('price_bin', observed=False).agg({
        'price_change': 'mean', 
        'airbnb_density': 'median'
    }).reset_index()
    
    london_X = london_bin_stats['price_change'].values.reshape(-1, 1)
    london_y = london_bin_stats['airbnb_density'].values
    
    london_poly = PolynomialFeatures(degree=london_best_deg, include_bias=False)
    london_X_poly = london_poly.fit_transform(london_X)
    london_model = LinearRegression()
    london_model.fit(london_X_poly, london_y)
    
    # Calculate R² for London model
    london_y_pred = london_model.predict(london_X_poly)
    london_r2 = r2_score(london_y, london_y_pred)
    
    london_X_curve = np.linspace(london_df['price_change'].min(), london_df['price_change'].max(), 100).reshape(-1, 1)
    london_y_curve = london_model.predict(london_poly.transform(london_X_curve))
    
    axes[1].scatter(london_X, london_y, color=LONDON_COLOR, s=80, label='Bin Medians')
    axes[1].plot(london_X_curve, london_y_curve, color=LONDON_COLOR, linestyle='--', label=f'Degree {london_best_deg} Fit')
    
    # Add bin edge vertical lines for London
    london_bin_edges = pd.unique(london_df['price_bin'].cat.categories.right.values[:-1])
    for edge in london_bin_edges:
        axes[1].axvline(x=edge, color='gray', linestyle=':', alpha=0.5)
    
    axes[1].set_title(f"London: Degree {london_best_deg} Regression on Binned Data (R² = {london_r2:.3f})")
    axes[1].set_xlabel(f"Mean Price Change ({LONDON_END_YEAR}–{LONDON_START_YEAR})")
    axes[1].set_ylabel("Median Airbnb Density (listings/km²)")
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig, axes

# For correlation bar chart
def plot_correlation_bar_chart(paris_corr, london_corr, paris_pvalue, london_pvalue):
    """Plot a bar chart comparing Pearson correlations for Paris and London with p-value annotations."""
    plt.figure(figsize=(10, 6))
    bars = plt.bar(
        ['Paris', 'London'],
        [paris_corr, london_corr],
        color=[PARIS_COLOR, LONDON_COLOR],
        edgecolor='black',
        alpha=0.85
    )
    plt.axhline(y=0, color='red', linestyle='--', alpha=0.4, linewidth=1)
    plt.ylim(-1, 1)
    plt.title('Correlation between Rental Price Changes and Airbnb Density', fontsize=16, fontweight='bold')
    plt.ylabel('Pearson Correlation Coefficient', fontsize=13)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.grid(axis='y', alpha=0.3, linestyle=':')
    
    # Annotate correlation values on bars
    for idx, (corr, pval) in enumerate(zip([paris_corr, london_corr], [paris_pvalue, london_pvalue])):
        plt.text(
            idx, corr + (0.07 if corr > 0 else -0.13),
            f"r={corr:.2f}\np={pval:.4f}",
            ha='center', va='bottom' if corr > 0 else 'top',
            fontsize=11, fontweight='bold',
            bbox=dict(boxstyle="round,pad=0.2", fc="white", ec="gray", alpha=0.7)
        )

    plt.tight_layout()

def plot_london_price_comparisons(london_data, london_best_deg):
    """
    Plot side-by-side comparison of:
    1. Airbnb density vs price change (polynomial regression)
    2. Airbnb density vs latest rental price (linear regression)
    
    Parameters:
    -----------
    london_data : GeoDataFrame
        London data containing 'airbnb_density', 'price_change', and latest price columns
    london_best_deg : int
        Best polynomial degree from cross-validation for London
    """
    fig, axes = plt.subplots(1, 2, figsize=(18, 8))
    
    # LEFT PLOT: Airbnb density vs price change (polynomial regression)
    london_X1 = london_data['price_change'].values.reshape(-1, 1)
    london_Y = london_data['airbnb_density'].values
    
    # Polynomial regression for price change
    london_poly = PolynomialFeatures(degree=london_best_deg, include_bias=False)
    london_X1_poly = london_poly.fit_transform(london_X1)
    london_model1 = LinearRegression()
    london_model1.fit(london_X1_poly, london_Y)
    
    # Calculate R² and R for polynomial model
    london_y_pred1 = london_model1.predict(london_X1_poly)
    london_r2_1 = r2_score(london_Y, london_y_pred1)
    london_r1 = np.sqrt(london_r2_1) if london_r2_1 >= 0 else 0
    
    # Plot scatter and regression line
    axes[0].scatter(london_X1, london_Y, alpha=0.7, color=LONDON_COLOR)
    
    # Create smooth curve for polynomial fit
    x_range = np.linspace(london_X1.min(), london_X1.max(), 100).reshape(-1, 1)
    y_pred = london_model1.predict(london_poly.transform(x_range))
    axes[0].plot(x_range, y_pred, color=LONDON_COLOR, linestyle='--', 
             label=f'Polynomial Regression (deg {london_best_deg}, R = {london_r1:.2f})')
    
    # Set labels and title
    axes[0].set_xlabel(f'Rental Price Change ({LONDON_END_YEAR}–{LONDON_START_YEAR})')
    axes[0].set_ylabel('Airbnb Density (listings/km²)')
    axes[0].set_title('London: Airbnb Density vs. Price Change')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # RIGHT PLOT: Airbnb density vs latest rental price (linear regression)
    london_X2 = london_data[f'avg_price_{LONDON_END_YEAR}'].values.reshape(-1, 1)
    
    # Linear regression for latest price
    london_model2 = LinearRegression()
    london_model2.fit(london_X2, london_Y)
    
    # Calculate R² and R for linear model
    london_r2_2 = r2_score(london_Y, london_model2.predict(london_X2))
    london_r2 = np.sqrt(london_r2_2) if london_r2_2 >= 0 else 0
    
    # Plot scatter and regression line
    axes[1].scatter(london_X2, london_Y, alpha=0.7, color=LONDON_COLOR)
    
    x_range = np.linspace(london_X2.min(), london_X2.max(), 100).reshape(-1, 1)
    y_pred = london_model2.predict(x_range)
    axes[1].plot(x_range, y_pred, color=LONDON_COLOR, linestyle='--', 
             label=f'Linear Regression (R = {london_r2:.2f})')
    
    # Set labels and title
    axes[1].set_xlabel(f'Rental Price ({LONDON_END_YEAR})')
    axes[1].set_ylabel('Airbnb Density (listings/km²)')
    axes[1].set_title(f'London: Airbnb Density vs. Latest Price ({LONDON_END_YEAR})')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig, axes

In [ ]:
# ----------------------------------------------------------------------
# DATA LOADING & PROCESSING
# ----------------------------------------------------------------------

In [ ]:
def load_paris_data():
    print("\n" + "="*50)
    print("LOADING PARIS DATA")
    print("="*50)

    # File paths
    csv_rentals = "../data/paris/paris_rentals.csv"        # FETCH FROM : https://www.data.gouv.fr/fr/datasets/logement-encadrement-des-loyers/#:~:text=Ce%20jeu%20de%20donn%C3%A9es%20pr%C3%A9sente,des%20ann%C3%A9es%20pr%C3%A9c%C3%A9dentes%20est%20conserv%C3%A9
    csv_airbnb = "../data/paris/paris_airbnb.csv"           # Airbnb listings : # FETCH FROM insideairbnb.com

    df_rentals_initial = pd.read_csv(csv_rentals, 
                                     delimiter=';', 
                                     on_bad_lines='skip', 
                                     encoding='utf-8')
    
    # Use fine grid neighborhoods from rentals data (geojson is given)
    df_neigh = df_rentals_initial.drop_duplicates(subset="Numéro du quartier")
    df_neigh = df_neigh[["Numéro du quartier", "geo_shape"]]
    print(f"Number of unique Paris neighborhoods: {len(df_neigh)}")
    
    # Rename and convert the GeoJSON geometry to Shapely objects
    df_neigh.rename(columns={"Numéro du quartier": "neigh_id"}, inplace=True)
    df_neigh["geometry"] = df_neigh["geo_shape"].apply(convert_geojson_to_shape)
    gdf_neigh = gpd.GeoDataFrame(df_neigh, geometry="geometry", crs="EPSG:4326")
    
    # Load Paris Airbnb data   :  # FETCH FROM insideairbnb.com
    df_airbnb = pd.read_csv(csv_airbnb, 
                            delimiter=',', 
                            on_bad_lines='skip', 
                            encoding='utf-8')
    
    # Create point geometry from latitude and longitude
    df_airbnb['geometry'] = df_airbnb.apply(lambda row: create_point_from_coords(row, 7, 6), axis=1)
    df_airbnb = df_airbnb[df_airbnb['geometry'].notnull()]
    gdf_airbnb = gpd.GeoDataFrame(df_airbnb, geometry='geometry', crs="EPSG:4326")
    
    # Spatial join: assign Airbnb listings to neighborhoods
    gdf_airbnb_joined = gpd.sjoin(gdf_airbnb, gdf_neigh, how='left', predicate='within')
    airbnb_counts = gdf_airbnb_joined.groupby('neigh_id').size().reset_index(name='airbnb_count')
    gdf_neigh = gdf_neigh.merge(airbnb_counts, on='neigh_id', how='left')
    gdf_neigh['airbnb_count'] = gdf_neigh['airbnb_count'].fillna(0).astype(int)
    
    # Compute area in km² and calculate density
    gdf_neigh['area_km2'] = gdf_neigh.to_crs(epsg=3857).area / 1e6
    gdf_neigh['airbnb_density'] = gdf_neigh['airbnb_count'] / gdf_neigh['area_km2']
    print(f"Total number of Airbnb listings in Paris: {len(gdf_airbnb)}")
    
    # Filter rental data by year
    df_rentals_2024 = df_rentals_initial[df_rentals_initial.iloc[:, 0] == 2024].copy()
    df_rentals_2019 = df_rentals_initial[df_rentals_initial.iloc[:, 0] == 2019].copy()
    
    # Parse coordinates and rental prices
    df_rentals_2024['geometry'] = df_rentals_2024.iloc[:, 13].apply(parse_paris_coords)
    df_rentals_2019['geometry'] = df_rentals_2019.iloc[:, 13].apply(parse_paris_coords)
    
    # Rental price is in column index 7
    df_rentals_2024['rental_price'] = pd.to_numeric(df_rentals_2024.iloc[:, 7], errors='coerce')
    df_rentals_2019['rental_price'] = pd.to_numeric(df_rentals_2019.iloc[:, 7], errors='coerce')
    
    # Filter valid entries
    df_rentals_2024 = df_rentals_2024[df_rentals_2024['geometry'].notnull() & df_rentals_2024['rental_price'].notnull()]
    df_rentals_2019 = df_rentals_2019[df_rentals_2019['geometry'].notnull() & df_rentals_2019['rental_price'].notnull()]
    
    # Create GeoDataFrames in preparation for spatial join
    gdf_rentals_2024 = gpd.GeoDataFrame(df_rentals_2024, geometry='geometry', crs="EPSG:4326")
    gdf_rentals_2019 = gpd.GeoDataFrame(df_rentals_2019, geometry='geometry', crs="EPSG:4326")
    
    # Spatial join rentals to neighborhoods
    gdf_rentals_2024_joined = gpd.sjoin(gdf_rentals_2024, gdf_neigh, how='left', predicate='within')
    gdf_rentals_2019_joined = gpd.sjoin(gdf_rentals_2019, gdf_neigh, how='left', predicate='within')
    
    # Calculate average prices for each neighborhood
    avg_prices_2024 = gdf_rentals_2024_joined.groupby('neigh_id')['rental_price'].mean().reset_index(name='avg_rental_price_2024')
    gdf_neigh = gdf_neigh.merge(avg_prices_2024, on='neigh_id', how='left')
    gdf_neigh['avg_rental_price_2024'] = gdf_neigh['avg_rental_price_2024'].fillna(0)
    
    avg_prices_2019 = gdf_rentals_2019_joined.groupby('neigh_id')['rental_price'].mean().reset_index(name='avg_rental_price_2019')
    gdf_neigh = gdf_neigh.merge(avg_prices_2019, on='neigh_id', how='left')
    gdf_neigh['avg_rental_price_2019'] = gdf_neigh['avg_rental_price_2019'].fillna(0)
    
    # Compute rental price increase (2024 - 2019)
    gdf_neigh['price_increase'] = gdf_neigh['avg_rental_price_2024'] - gdf_neigh['avg_rental_price_2019']
    
    return gdf_neigh

In [ ]:
def load_london_data():
    print("\n" + "="*50)
    print("LOADING LONDON DATA")
    print("="*50)
    
    # File paths
    excel_rentals = "../data/london/london_rentals.xls"        # rentals data : # FETCH FROM : https://data.london.gov.uk/dataset/average-private-rents-borough
    csv_airbnb = "../data/london/london_airbnb.csv"           # Airbnb listings : # FETCH FROM insideairbnb.com
    geojson_neigh = "../data/london/london_neighbourhoods.geojson"  # neighbourhood shapes
    
    # Load Airbnb listings
    df_airbnb = pd.read_csv(csv_airbnb, encoding="utf-8")
    
    # Build GeoDataFrame for Airbnb data
    DF = df_airbnb.copy()
    DF['geometry'] = DF.apply(lambda row: create_point_from_coords(row, 7, 6), axis=1)
    DF = DF[DF['geometry'].notnull()]
    gdf_airbnb = gpd.GeoDataFrame(DF, geometry='geometry', crs='EPSG:4326')
    
    # Load and process rental data
    # positional column indices in the Excel
    YEAR_COL, QUARTER_COL, NEIGH_COL, CATEGORY_COL, PRICE_COL = 0, 1, 3, 4, 6
    NEIGH_NAME = 'neighbourhood'
    
    # read raw rentals sheet
    raw = pd.read_excel(excel_rentals, sheet_name="Raw data", header=None)
    raw.rename(columns={NEIGH_COL: NEIGH_NAME}, inplace=True)
    
    # filter by years, Q1, all categories
    df_filt = raw[(raw.iloc[:, YEAR_COL].isin([LONDON_START_YEAR, LONDON_END_YEAR])) &
                   (raw.iloc[:, QUARTER_COL]=='Q1') &
                   (raw.iloc[:, CATEGORY_COL]=='All categories')].copy()
    
    # parse price and drop NAs
    df_filt[PRICE_COL] = pd.to_numeric(df_filt.iloc[:, PRICE_COL], errors='coerce')
    df_filt.dropna(subset=[NEIGH_NAME, PRICE_COL], inplace=True)
    
    # average price per neighbourhood per year
    avg_start = df_filt[df_filt.iloc[:, YEAR_COL]==LONDON_START_YEAR]
    avg_start = avg_start.groupby(NEIGH_NAME)[PRICE_COL].mean().reset_index(name=f"avg_price_{LONDON_START_YEAR}")
    avg_end = df_filt[df_filt.iloc[:, YEAR_COL]==LONDON_END_YEAR]
    avg_end = avg_end.groupby(NEIGH_NAME)[PRICE_COL].mean().reset_index(name=f"avg_price_{LONDON_END_YEAR}")
    
    # merge average prices
    df_rentals = pd.merge(avg_start, avg_end, on=NEIGH_NAME, how='outer').fillna(0)
    
    # compute price change always
    df_rentals['price_change'] = df_rentals[f"avg_price_{LONDON_END_YEAR}"] - df_rentals[f"avg_price_{LONDON_START_YEAR}"]
    
    # Merge with neighbourhood geometries and compute Airbnb density
    gdf_neigh = gpd.read_file(geojson_neigh)
    gdf_neigh = gdf_neigh.merge(df_rentals, on=NEIGH_NAME, how='left').fillna(0)
    joined = gpd.sjoin(gdf_airbnb, gdf_neigh, how='left', predicate='within')
    counts = joined.groupby('neighbourhood_right').size().reset_index(name='airbnb_count')
    counts.rename(columns={'neighbourhood_right':'neighbourhood'}, inplace=True)
    gdf_neigh = gdf_neigh.merge(counts, on='neighbourhood', how='left').fillna({'airbnb_count':0})
    gdf_neigh['area_km2'] = gdf_neigh.to_crs(epsg=3857).area / 1e6
    gdf_neigh['airbnb_density'] = gdf_neigh['airbnb_count'] / gdf_neigh['area_km2']
    
    print(f"Number of unique London neighborhoods: {len(gdf_neigh)}")
    print(f"Total Airbnb listings in London: {len(gdf_airbnb)}")
    
    return gdf_neigh

In [ ]:
# ----------------------------------------------------------------------
# COMPARATIVE ANALYSIS
# ----------------------------------------------------------------------

In [ ]:
# Load data for both cities
paris_data = load_paris_data()
london_data = load_london_data()
print("\nOK")

&nbsp;
## What are the Airbnb densities in Paris & London ?
&nbsp;

In [ ]:
# 1. Airbnb Density Maps
fig, axes = plt.subplots(1, 2, figsize=(18, 8))

paris_data.plot(column='airbnb_density', cmap='Blues', legend=True, ax=axes[0], edgecolor='black')
axes[0].set_title('Paris: Airbnb Density (listings/km²)')

london_data.plot(column='airbnb_density', cmap='Greens', legend=True, ax=axes[1], edgecolor='black')
axes[1].set_title('London: Airbnb Density (listings/km²)')

plt.tight_layout()
plt.show()

&nbsp;
## What are the long term rental price increase in Paris & London ?
&nbsp;

In [ ]:
# 2. Rental Price Increase Maps
fig, axes = plt.subplots(1, 2, figsize=(18, 8))

paris_data.plot(column='price_increase', cmap='Blues', legend=True, ax=axes[0], edgecolor='black')
axes[0].set_title('Paris: Rental Price Increase (2024 - 2019)')

london_data.plot(column='price_change', cmap='Greens', legend=True, ax=axes[1], edgecolor='black')
axes[1].set_title(f'London: Rental Price Change ({LONDON_END_YEAR}–{LONDON_START_YEAR})')

plt.tight_layout()
plt.show()

&nbsp;
# Data Exploration

## Market Structure

In [ ]:
# 3. Density Distribution Comparison
fig, ax = plot_density_distribution_comparison(paris_data, london_data)
plt.show()

Right-skewed distributions suggest a few neighborhoods with very high Airbnb concentration, which might represent two distinct type of neighbourhoods : tourist vs residential)
&nbsp;
## The plot twist
&nbsp;

In [ ]:
# 4. Scatter Plot: Airbnb Density vs Price Increase
fig, axes = plot_airbnb_density_scatter_comparison(paris_data, london_data)
plt.show()

&nbsp;
## Let's do a simple polynomial regression
### With cross validation to find the best degree of approximation
&nbsp;

In [ ]:
# 5. Bias-Variance Trade-off Analysis
paris_X = paris_data['price_increase'].values.reshape(-1, 1)
paris_Y = paris_data['airbnb_density'].values
paris_cv_results = fit_polynomial_models(paris_X, paris_Y)
print("\nParis Cross-validation results:")
print(paris_cv_results)

london_X = london_data['price_change'].values.reshape(-1, 1)
london_Y = london_data['airbnb_density'].values
london_cv_results = fit_polynomial_models(london_X, london_Y)
print("\nLondon Cross-validation results:")
print(london_cv_results)

fig, axes = plot_bias_variance_tradeoff_comparison(paris_cv_results, london_cv_results)
plt.show()

&nbsp;
## What is the best polynomial degree to fit theses data points ?
&nbsp;

In [ ]:
# Fit best models for both Paris and London
paris_best_deg, paris_poly, paris_model, paris_r = fit_best_model(paris_X, paris_Y, paris_cv_results)
london_best_deg, london_poly, london_model, london_r = fit_best_model(london_X, london_Y, london_cv_results)

print(f"Best Polynomial Degree for Paris : {paris_best_deg}")
print(f"Best Polynomial Degree for London : {london_best_deg}")
# 6. Plot polynomial regression comparison
fig, axes = plot_polynomial_regression_comparison(
    paris_X, paris_Y, paris_poly, paris_model, paris_r, paris_best_deg,
    london_X, london_Y, london_poly, london_model, london_r, london_best_deg
)
plt.show()

&nbsp;
## Can we get a better view of this tends ?
### Using a box plot
&nbsp;

In [ ]:
# 7. Box Plots - with 5 bins
paris_data['price_increase_bin'] = pd.qcut(paris_data['price_increase'], q=5, duplicates='drop')
london_data['price_bin'] = pd.qcut(london_data['price_change'], q=5, duplicates='drop')

plot_boxplot_comparison(paris_data, london_data)
plt.show()

&nbsp;
## And an even better ?
### By plotting the median of each bins of the box plots
&nbsp;

In [ ]:
# 8. Quadratic Regression on Binned Data
plot_quadratic_fit_comparison(paris_data, london_data, paris_best_deg, london_best_deg)
plt.show()

&nbsp;
## What are the correlation ?
### By using pearson correlation
The Pearson correlation coefficient encapsulates in a single value between –1 and 1 the strength and direction of a straight‑line relationship: +1 denotes perfect positive alignment, –1 perfect negative alignment, and 0 no linear connection.
&nbsp;

In [ ]:
# 9. Pearson Correlation Comparison
paris_corr, paris_pvalue = pearsonr(paris_data['price_increase'], paris_data['airbnb_density'])
london_corr, london_pvalue = pearsonr(london_data['price_change'], london_data['airbnb_density'])

# Create correlation bar chart
plot_correlation_bar_chart(paris_corr, london_corr, paris_pvalue, london_pvalue)
plt.show()

print(f"Paris - Correlation between Price Increase and Airbnb Density: {paris_corr:.3f} (p-value: {paris_pvalue:.4f})")
print(f"London - Correlation between Price Change and Airbnb Density: {london_corr:.3f} (p-value: {london_pvalue:.4f})")

&nbsp;
### Optional observation
&nbsp;

In [ ]:
# 10. London: Price Change vs Latest Price Comparison
fig, axes = plot_london_price_comparisons(london_data, london_best_deg)
plt.show()
print(secret_observation)

# Open-ended Challenge
## a. Data to gather

1. **Macroeconomic indicators**  
   - GDP growth and unemployment rates (city/metro level)  
   - Average wage or income growth  

2. **Housing supply metrics**  
   - Building permits, housing completions, vacancy rates  
   - Land‑use policy changes (zoning amendments, rent‑control legislation)  

3. **Demographic and migration flows**  
   - Net migration (domestic + international)  
   - Household formation rates  

4. **Tourism and short‑term rentals**  
   - Airbnb/VRBO listings over time and occupancy rates  
   - Hotel capacity and visitor arrivals  

5. **Financial conditions**  
   - Mortgage rates and credit availability  
   - Investor purchase volumes in residential real estate  

---

## b. Investigation and analytical structure

1. **Define the incremental difference**  
   - Compute Δ<sub>city</sub> = Rent<sub>t₁</sub> – Rent<sub>t₀</sub> for each city  
   - Compare Δ<sub>City A</sub> – Δ<sub>City B</sub>  

2. **Formulate hypotheses**  
   - e.g. “City A’s higher rent growth is driven by tighter new supply”  

3. **Data alignment & preprocessing**  
   - Align time windows (e.g. annual Q1), geography (neighborhoods), and units (€/m² vs £/ft²)  
   - Normalize variables (z‑scores) for comparability  

4. **Descriptive & exploratory analysis**  
   - Time‑series plots of candidate drivers vs. rent indices  
   - Spatial maps showing rent increases and supply/demand indicators  

5. **Multivariate regression**  
   ```text
   RentChange<sub>i,t</sub> = β₀ + β₁·SupplyGrowth<sub>i,t</sub> + β₂·IncomeGrowth<sub>i,t</sub> + β₃·AirbnbDensity<sub>i,t</sub> + … + ε<sub>i,t</sub>
